In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
PATH_TO_RESULTS = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_analysis/'
PATH_TO_PLOTS = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_analysis/plots/'

In [3]:
waveform = 'IMRPhenomXPHM'

In [4]:
estimator = 'maxP'
events = np.loadtxt(PATH_TO_RESULTS + 'info/' + 'events_with_priors_%s.txt' %waveform, dtype=str)
with open(PATH_TO_RESULTS + 'info/' + 'detectors_dictionary.pkl', 'rb') as f:
    detectors = pickle.load(f)

In [5]:
params = ['chirp_mass', 'mass_ratio', 'luminosity_distance', 'dec', 'ra', 'theta_jn', 'psi', 'phase', 
            'geocent_time', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl']

In [6]:
my_colors = [(1.0, 0.4980392156862745, 0.054901960784313725), #orange
            (0.12156862745098039, 0.4666666666666667, 0.7058823529411765)] #blue

In [7]:
for event in events:

    injections = pd.read_hdf(PATH_TO_RESULTS + 'injections/' + event + '/%s_%s_%s.hdf5' %(event, waveform, estimator))
    lvk_samples = pd.read_hdf(PATH_TO_RESULTS + 'results/samples/' + 'lvk_samples_%s.hdf5' %event)
    fisher_samples = pd.read_hdf(PATH_TO_RESULTS + 'results/samples/' + 'fisher_samples_%s_%s.hdf5' %(estimator, event))
    fisher_plus_priors_samples = pd.read_hdf(PATH_TO_RESULTS + 'results/samples/' + 'fisher_plus_priors_samples_%s_%s.hdf5' %(estimator, event))

    fig, axs = plt.subplots(5, 3, figsize=(15, 10))

    for i, param in enumerate(params):
        ax = axs[i//3, i%3]
        ax = sns.histplot(lvk_samples[param], element='step', ax=ax, label='LVK', alpha = 0.5, color = 'black')
        ax.axvline(injections[param].values[0], color='red', label = 'injection', linewidth=1.5)
        ax = sns.histplot(fisher_samples[param], element='step', ax=ax, label='Fisher', alpha = 0.25, color = my_colors[0])
        ax = sns.histplot(fisher_plus_priors_samples[param], element='step', ax=ax, label='Fisher + priors', alpha=0.5, color = my_colors[1])
        
        ax.set_ylabel('Count', fontsize=10)
        ax.set_xlabel('%s' %param, fontsize=10)
        ax.set_yscale('log')
        ax.tick_params(axis='both', which='major', labelsize=8)
        ax.tick_params(axis='both', which='minor', labelsize=8)
        ax.legend(fontsize=8)

    plt.suptitle('%s' %event, fontsize=15)
    plt.tight_layout()
    plt.savefig(PATH_TO_PLOTS + 'samples_%s/samples_comparison_%s_%s.png' %(estimator, estimator, event), dpi=300)
    plt.close()